In [1]:
from requests import get
import pandas as pd
from API_Keys import quandl
from datetime import datetime, timedelta

# To add new data sources from Quandl
Add your appropriately named URL in the cell below to the right group.

Add that variable to the URL list at the end of the correct group

Adjust range in cells that both merge them and also filter them to account for your addition

For the renaming of merged columns, add your renaming schemata to the correct area.

# This will fail if any step is messed up, or if the quandl data is organized differently than expected. This is hard to automate, potential issues can be expected.

Also, if we need to keep different columns for new data, this will also break.

In [2]:
# Group 1, keep Change, Settle, Volume, Previous Day Open Interest
CMEgold_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_GC1.json?api_key={quandl}"
CMEsilver_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_SI1.json?api_key={quandl}"
CMEpalladium_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_PA1.json?api_key={quandl}"
CMEplatinum_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_PL1.json?api_key={quandl}"
CME_TenYear_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_TY1.json?api_key={quandl}"

#Group 2, keep Change, Settle, Wave, Volume, Prev. Day Open Interest
ICE_USD_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/ICE_DX1.json?api_key={quandl}"
ICE_ZAR_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/ICE_ZR1.json?api_key={quandl}"
# SGX_EuroD_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/SGX_ED1.json?api_key={quandl}"

#Group 3, keep Open Interest, last one on list also decides cut off date.
# Last one needs to be shortest weekly separation
Gold_OpenInt = f"https://www.quandl.com/api/v3/datasets/CFTC/088691_FO_ALL.json?api_key={quandl}"
Silver_OpenInt = f"https://www.quandl.com/api/v3/datasets/CFTC/084691_FO_ALL.json?api_key={quandl}"
Palladium_OpenInt = f"https://www.quandl.com/api/v3/datasets/CFTC/075651_FO_ALL.json?api_key={quandl}"
Platinum_OpenInt = f"https://www.quandl.com/api/v3/datasets/CFTC/076651_FO_ALL.json?api_key={quandl}"

In [11]:
url_list = [CMEgold_futures,CMEsilver_futures,CMEpalladium_futures,CMEplatinum_futures,CME_TenYear_futures,ICE_USD_futures,ICE_ZAR_futures,Gold_OpenInt,Silver_OpenInt,Palladium_OpenInt,Platinum_OpenInt]

In [12]:
info_list = [get(url).json() for url in url_list]

In [26]:
df_list = [pd.DataFrame(info["dataset"]["data"],columns=info["dataset"]["column_names"]) for info in info_list]

In [28]:
# Creates a list that functions well in testing
pruned_list = [None]*11

In [29]:
# Cell to prune Group 1
metal_namer = {0:"Gold",1:"Silver",2:"Palladium",3:"Platinum",4:"Ten_Yr_Futures"}
for i in range(5):
    pruned_list[i] = df_list[i][["Date","Change","Settle","Volume","Previous Day Open Interest"]]
    pruned_list[i].columns = ["Date",f"{metal_namer[i]}_Change",f"{metal_namer[i]}_Settle",f"{metal_namer[i]}_Volume",f"{metal_namer[i]}_Prev. Day Open Interest"]

In [30]:
# Cell to prune Group 2
money_namer = {5:"USD",6:"ZAR"}
for i in range(5,7):
    pruned_list[i] = df_list[i][["Date","Change","Settle","Volume","Wave","Prev. Day Open Interest"]]
    pruned_list[i].columns = ["Date",f"{money_namer[i]}_Change",f"{money_namer[i]}_Settle",f"{money_namer[i]}_Volume",f"{money_namer[i]}_Wave",f"{money_namer[i]}_Prev. Day Open Interest"]

In [31]:
for i in range(7,11):
    pruned_list[i] = df_list[i][["Date","Open Interest"]]

In [32]:
# Only works for days that are Tuesday releases on their latest and most recent
# will need to add code to adjust for late releases because of Holidays
cut_off_date = df_list[len(df_list)-1]["Date"][len(df_list[len(df_list)-1].index)-1]
cut_off_date = (datetime.strptime(cut_off_date,"%Y-%m-%d") - timedelta(days=6))
cut_off_date = cut_off_date.strftime("%Y-%m-%d")
# recent_off_date = df_list[len(df_list)-1]["Date"][0]
# recent_off_date = (datetime.strptime(recent_off_date,"%Y-%m-%d") - timedelta(days=4))
# recent_off_date = recent_off_date.strftime("%Y-%m-%d")

In [33]:
for i in range(7):
    pruned_list[i] = pruned_list[i].loc[pruned_list[i]["Date"]>=cut_off_date]
#     pruned_list[i] = pruned_list[i].loc[pruned_list[i]["Date"]<=recent_off_date]

In [34]:
tuple_list = [((datetime.strptime(cut_off_date,"%Y-%m-%d")+timedelta(days=(w*7))),(datetime.strptime(cut_off_date,"%Y-%m-%d")+timedelta(days=(w*7 + 4)))) for w in range(len(df_list[len(df_list)-1].index))]
cut_bins = pd.IntervalIndex.from_tuples(tuple_list,closed="both")

In [35]:
for bin_candidate in range(7):
    bins = pd.cut(pruned_list[bin_candidate]["Date"],cut_bins)
    pruned_list[bin_candidate] = pruned_list[bin_candidate].groupby(bins).agg()
    pruned_list[bin_candidate] = pruned_list[bin_candidate].assign(Date=list(pruned_list[len(pruned_list)-1]["Date"].iloc[::-1]))
    pruned_list[bin_candidate].index.names = ["Business_Week"]

In [36]:
merged_list = [pruned_list[i].\
    merge(pruned_list[7+i], on = "Date", how = "inner").\
    merge(pruned_list[4], on = "Date", how = "inner").\
    merge(pruned_list[5], on = "Date", how = "inner").\
    merge(pruned_list[6], on = "Date", how = "inner").\
    set_index(pruned_list[i].index) for i in range(4)]

In [38]:
merged_list[0].tail(62)

,Gold_Change,Gold_Settle,Gold_Volume,Gold_Prev. Day Open Interest,Date,Open Interest,Ten_Yr_Futures_Change,Ten_Yr_Futures_Settle,Ten_Yr_Futures_Volume,Ten_Yr_Futures_Prev. Day Open Interest,USD_Change,USD_Settle,USD_Volume,USD_Wave,USD_Prev. Day Open Interest,ZAR_Change,ZAR_Settle,ZAR_Volume,ZAR_Wave,ZAR_Prev. Day Open Interest
Business_Week,,,,,,,,,,,,,,,,,,,,
"[2018-05-09, 2018-05-13]",4.000000,1317.033333,27.000000,284.000000,2018-05-15,740326.0,0.130208,119.354167,1.148134e+06,3.712667e+06,-0.182333,92.603667,27155.000000,92.680667,34910.000000,-0.106500,12.427833,15.000000,12.441080,318.333333
"[2018-05-16, 2018-05-20]",1.766667,1289.533333,10.000000,99.000000,2018-05-22,730741.0,0.140625,118.635417,1.594402e+06,3.681246e+06,0.150333,93.400000,23111.666667,93.367667,37523.333333,0.060667,12.650250,7.000000,12.658750,317.000000
"[2018-05-23, 2018-05-27]",5.733333,1298.766667,3.666667,20.000000,2018-05-29,670827.0,0.328125,119.583333,3.028707e+06,3.022577e+06,0.201333,93.916333,24777.000000,93.851667,38682.333333,-0.025000,12.487000,46.000000,12.522930,312.000000
"[2018-05-30, 2018-06-03]",3.066667,1298.800000,18174.000000,18268.333333,2018-06-05,648436.0,0.437500,120.505208,8.726620e+05,3.764080e+05,-0.206667,94.081000,32922.000000,94.135333,41344.666667,-0.006667,12.665500,3.666667,12.637083,304.666667
"[2018-06-06, 2018-06-10]",0.866667,1297.966667,367.333333,5342.666667,2018-06-12,651671.0,0.351562,119.822917,2.860233e+04,5.266567e+04,-0.111333,93.515667,20748.333333,93.532667,38817.000000,0.098917,12.953583,5.500000,12.981625,258.000000
"[2018-06-13, 2018-06-17]",12.766667,1291.833333,255.333333,605.666667,2018-06-19,691070.0,0.197917,119.718750,9.146000e+03,2.196600e+04,0.327000,94.419000,27871.333333,94.234667,27182.000000,0.026750,13.398833,20.333333,13.328020,223.000000
"[2018-06-20, 2018-06-24]",2.866667,1268.600000,42.333333,202.666667,2018-06-26,664598.0,0.151042,119.885417,7.666060e+05,2.308246e+06,-0.183000,94.433333,20947.333333,94.611000,45799.000000,-0.105583,13.692750,23.000000,13.739533,290.000000
"[2018-06-27, 2018-07-01]",4.133333,1252.766667,153217.000000,213218.333333,2018-07-03,714279.0,0.296875,120.255208,1.490481e+06,3.480312e+06,0.001000,94.804667,22653.666667,94.713667,47111.666667,0.066417,13.947417,25.666667,13.947960,293.666667
"[2018-07-04, 2018-07-08]",4.150000,1257.300000,284468.000000,316741.500000,2018-07-10,719736.0,0.078125,120.296875,1.030142e+06,3.577026e+06,-0.214333,94.134667,14843.000000,94.094333,47512.666667,-0.099000,13.713833,46.666667,13.723407,251.666667


In [40]:
list(merged_list[0].columns)

['Gold_Change',
 'Gold_Settle',
 'Gold_Volume',
 'Gold_Prev. Day Open Interest',
 'Date',
 'Open Interest',
 'Ten_Yr_Futures_Change',
 'Ten_Yr_Futures_Settle',
 'Ten_Yr_Futures_Volume',
 'Ten_Yr_Futures_Prev. Day Open Interest',
 'USD_Change',
 'USD_Settle',
 'USD_Volume',
 'USD_Wave',
 'USD_Prev. Day Open Interest',
 'ZAR_Change',
 'ZAR_Settle',
 'ZAR_Volume',
 'ZAR_Wave',
 'ZAR_Prev. Day Open Interest']

In [ ]:
merged_list[0].to_csv